# 05-Supervised learning 

In this notebook we will train a model based on the previous 4 notebooks. This include the use of two supervised classifiers named RandomForest and Gradient Boost. We will be using the library of scikitlearn to do this. However, there is no limitation in number of algorithms that can be used, since the notebook has been made modular for further easy implementations of other machine learning algorithms. 

## Table of contents

- Imports 
- Algorithms
- Dataset analysis
    - How do we evaluate the model?
    - Optimal hyperparameter search 
- Running the different algorithms
- Visualizing the cross-validated trained models
    - Interpreting important features used by the models
    - Precision and recall metrics
- Predicting solid-state qubit candidates
    - Save the summary and models

In [1]:
# Optional: Load the "autoreload" extension so that code can change
%load_ext autoreload

#OPTIONAL: Always reload modules so that as you change code in src, it gets loaded
%autoreload 2

# Imports

In [2]:
import sys
sys.path.insert(0, "../")

from pathlib import Path
data_dir = Path.cwd().parent.parent / "data"
models_dir = Path.cwd().parent.parent / "models" 

print("Current data directory {}".format(data_dir))

Current data directory /home/oliver/Dokumenter/masterprosjekt/predicting-solid-state-qubit-candidates/data


In [3]:
from src.models import train_model, predict_model
from src.features import build_features
from src.visualization import visualize


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



In [4]:
#Standard libraries
import numpy as np
import pandas as pd
import pickle

#Models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#from sklearn.neural_network import MLPClassifier

#Feature selections
from sklearn.model_selection import RepeatedStratifiedKFold

#metrics and nice visualization
from tqdm import tqdm

#visualizations
import plotly.graph_objs as go

# setting random seed for reproducibility
random_state=23462478

In [5]:
InsertApproach = "01-naive-approach"

In [6]:
trainingData   = pd.read_pickle(data_dir / InsertApproach / "processed" / "trainingData.pkl")
trainingTarget= pd.read_pickle(data_dir / InsertApproach / "processed" / "trainingTarget.pkl")
testSet       = pd.read_pickle(data_dir / InsertApproach / "processed" / "testSet.pkl")

trainingData

,material_id,AtomicOrbitals|HOMO_character,AtomicOrbitals|HOMO_element,AtomicOrbitals|HOMO_energy,AtomicOrbitals|LUMO_character,AtomicOrbitals|LUMO_element,AtomicOrbitals|LUMO_energy,AtomicOrbitals|gap_AO,AtomicPackingEfficiency|mean simul. packing efficiency,AtomicPackingEfficiency|mean abs simul. packing efficiency,...,OxidationStates|std_dev oxidation state,MP_Eg,OQMD_Eg,AFLOW_Eg,AFLOW-fitted_Eg,AFLOWML_Eg,JARVIS-TBMBJ_Eg,JARVIS-OPT_Eg,Exp_Eg,spillage
0,mp-7,2.0,16,-0.261676,2.0,16,-0.261676,0.000000,0.023994,0.023994,...,0.0,2.4881,2.085,2.5251,4.31683,2.490,3.0448,1.9604,0.0,0.000
1,mp-14,2.0,34,-0.245806,2.0,34,-0.245806,0.000000,0.023994,0.023994,...,0.0,1.0119,0.000,0.9784,2.23188,0.997,2.2888,0.8982,0.0,0.000
2,mp-19,2.0,52,-0.226594,2.0,52,-0.226594,0.000000,0.023994,0.023994,...,0.0,0.5752,0.000,0.1534,1.11978,0.000,0.6148,0.1655,0.0,1.318
3,mp-24,2.0,6,-0.199186,2.0,6,-0.199186,0.000000,0.023994,0.023994,...,0.0,2.7785,0.000,2.4528,4.21937,3.355,3.3186,2.7427,0.0,0.000
4,mp-47,2.0,6,-0.199186,2.0,6,-0.199186,0.000000,0.023994,0.023994,...,0.0,3.3395,0.000,3.3412,5.41694,3.166,4.7622,3.7645,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,mp-1205479,2.0,9,-0.415606,2.0,51,-0.185623,0.229983,0.000000,0.000000,...,0.0,4.7325,0.000,0.0000,0.00000,4.468,0.0000,0.0000,0.0,0.000
1614,mp-1208643,2.0,16,-0.261676,1.0,72,-0.166465,0.095211,-0.033747,0.037006,...,0.0,2.0461,0.000,0.0000,0.00000,1.346,0.0000,0.0000,0.0,0.000
1615,mp-1210722,2.0,8,-0.338381,2.0,52,-0.226594,0.111787,-0.007146,0.018646,...,0.0,3.2147,0.000,0.0000,0.00000,2.872,0.0000,0.0000,0.0,0.000
1616,mp-1232407,1.0,1,-0.233471,1.0,1,-0.233471,0.000000,0.000000,0.000000,...,0.0,5.9434,0.000,0.0000,0.00000,5.359,0.0000,0.0000,0.0,0.000


# Algorithms
Below we define the algorithm to use and its abbreviation. Parameters that are optional to tune are the parameters to the algorithms, with the default value as their optimised value. Another parameter to tune is how many cross-validations one wants to iterate through for the analysis. In addition, one has to find the best features for a new algorithm which will be added further down in the notebook.

In [7]:
InsertAlgorithms    = [RandomForestClassifier    (max_depth = 5, random_state=random_state),\
                 GradientBoostingClassifier(max_depth = 2, random_state=random_state)]
InsertAbbreviations = ["RF", "GB"]
InsertprettyNames   = ["Random Forest", "Gradient Boost"]

# Dataset analysis


## How do we evaluate the model?

### Cross-validation

Cross-validation is a technique to evaluate predictive models by partitioning the original sample into a training set to train the model, and a test set to evaluate it. 

### k-fold cross-validation

In k-fold cross-validation, the sample is partioned into k equal sized subsamples. Of the k samples, a single sample is used as validation set while the remaining k-1 samples are used as training data. The process is then repeated k-times, such that each of the k-th subsample is used as validation set exactly one time. Therefore, all observations are used for both training and validation, and each observation is used for validation exactly once. The k results from the folds can then be averaged to produce an estimate.

### Stratified k-fold cross-validation

In stratified k-fold cross validation, the fold that is selected contains roughly the same proportions of existing class labels. 

### n-repeated stratified k-fold cross-validation

In n-repeated stratified k-fold cross-validation, the stratified k-fold cross-validation is repeated n times, which yields n random partitions of the original sample. The n results can be averaged to produce a single estimation. 

## Sample size
To not discrimate a class, we make sure that each class is equally represented in the subsamples. Underneath shows a brief overview of the different methods involved to deal with this challenge.

### Random oversampling of minority class

Random oversampling can be achived by randomly duplicating examples from the minority class and adding them to the training dataset. 

The approach can be effective to algorithms that are vulnerable to a skewed dsitribution, however, it can also affect algorithms to overfit the minority class. 

### Random Undersampling of majority class

Random undersampling involves randomly selecting examples from the majority class to delete from the training dataset. 

This can prove problematic, since the loss of data can make the decision boundary between minority and majority instances harder to learn. Additionally, there is a chance that the model might loose valuable information. 

### Both oversampling and undersampling

A third option might be to combine the two of them. 


## Optimal hyperparameters search

In this section we will find the optimal parameters used for the various algorithms. We will use imblearn's Pipeline and its implemented samplers, such as SMOTE and RandomUnderSampler. The advantage of using imblearn instead of sklearn, is that sklearn's pipeline will fit the samplers to the validation data as well, while imblearn only fit the resamplers to the training data. We store the best estimators and use them again under this section.

In [ ]:
numberRuns=1
numberSplits = 10

includeSampleMethods = ["", "under", "over", "both"]

Abbreviations = []
prettyNames   = []
Algorithms = []

rskfold = RepeatedStratifiedKFold(n_splits=numberSplits, n_repeats=numberRuns, random_state=random_state)

ModelsBestParams = pd.Series({}, dtype="string")

for i, algorithm in tqdm(enumerate(InsertAlgorithms)):
    for method in includeSampleMethods:
        print("Finding best params for: {}".format(InsertAbbreviations[i] + " " + method))
        bestEstimator, ModelsBestParams[InsertAbbreviations[i] + " " + method] = train_model.applyGridSearch(
                                                                             X = trainingData.drop(["material_id", "full_formula"], axis=1), 
                                                                             y = trainingTarget.values.reshape(-1,),
                                                                        model = algorithm, 
                                                                           cv = rskfold, 
                                                                 sampleMethod = method)
        Abbreviations.append(InsertAbbreviations[i] + " " + method)
        prettyNames.append(InsertAbbreviations[i] + " " + method)
        Algorithms.append(bestEstimator)

0it [00:00, ?it/s]

Finding best params for: RF 
Fitting 10 folds for each of 96 candidates, totalling 960 fits
Finding best params for: RF under
Fitting 10 folds for each of 96 candidates, totalling 960 fits
Finding best params for: RF over
Fitting 10 folds for each of 96 candidates, totalling 960 fits
Finding best params for: RF both
Fitting 10 folds for each of 96 candidates, totalling 960 fits


1it [12:32, 752.77s/it]

Finding best params for: GB 
Fitting 10 folds for each of 36288 candidates, totalling 362880 fits


In [ ]:
pd.DataFrame(ModelsBestParams["RF "].cv_results_)

In [ ]:
plt.figure(figsize=(13, 13))
plt.title("GridSearchCV evaluating using multiple scorers simultaneously",
          fontsize=16)

plt.xlabel("min_samples_split")
plt.ylabel("Score")

ax = plt.gca()
ax.set_xlim(0, 402)
ax.set_ylim(0.73, 1)

# Get the regular numpy array from the MaskedArray
X_axis = np.array(ModelsBestParams["RF "].cv_results_['param_model__n_estimators'].data, dtype=float)

for scorer, color in zip(sorted(scoring), ['g', 'k']):
    for sample, style in (('train', '--'), ('test', '-')):
        sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
        sample_score_std = results['std_%s_%s' % (sample, scorer)]
        ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                        sample_score_mean + sample_score_std,
                        alpha=0.1 if sample == 'test' else 0, color=color)
        ax.plot(X_axis, sample_score_mean, style, color=color,
                alpha=1 if sample == 'test' else 0.7,
                label="%s (%s)" % (scorer, sample))

    best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
    best_score = results['mean_test_%s' % scorer][best_index]

    # Plot a dotted vertical line at the best score for that scorer marked by x
    ax.plot([X_axis[best_index], ] * 2, [0, best_score],
            linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)

    # Annotate the best score for that scorer
    ax.annotate("%0.2f" % best_score,
                (X_axis[best_index], best_score + 0.005))

plt.legend(loc="best")
plt.grid(False)
plt.show()

# Running the different supervised models
Under follows the general model runSupervisedModel that takes the as parameter which model to run and returns nice statistics formatted as a dictionary.

In [ ]:
SupervisedModels = pd.Series({}, dtype="string")

In [ ]:
for i, algorithm in enumerate(Algorithms): 
    print("Current training algorithm: {}".format(prettyNames[i]))
    SupervisedModels[Abbreviations[i]] = (
        train_model.runSupervisedModel(classifier  = algorithm, 
                                     X = trainingData.drop(["material_id", "full_formula"], axis=1), 
                                     y = trainingTarget.values.reshape(-1,),
                                     k = numberSplits,
                                     n = numberRuns,
                      resamplingMethod = "under",
                     featureImportance = True)
    )

# Visualizing the cross-validated trained models

In [ ]:
visualize.plot_accuracy(SupervisedModels, prettyNames)

The standard deviation is calculated as a function difference of the $100$ models in the purpose of visalizing how much the models deviate from each other.

## Interpreting important features used by the models

Which features are the most important in predicting to give a label $0$ or $1$?

In [ ]:
"""
def plot_important_features(models, names):
    fig = go.Figure( 
            layout = go.Layout (
                title=go.layout.Title(text='Features used in model (Nruns = {})'.format(numberRuns*numberSplits)),
                yaxis=dict(title="Number times"),
                barmode='group'
            )
        )

    for i, model in enumerate(models):
        fig.add_traces(go.Bar(name=names[i], x=trainingData.columns.values, y=model['importantKeys']))

    fig.show()

    fig = go.Figure( 
            layout = go.Layout (
                title=go.layout.Title(text="Feature Importance for the 100th iteration".format(numberRuns*numberSplits)),
                yaxis=dict(title='Relative importance'),
                barmode='group'
            )
        )

    for i, model in enumerate(models):
        fig.add_traces(go.Bar(name=names[i], x=trainingData.columns.values, y=model['relativeImportance']))

    fig.show()
"""
#visualize.plot_important_features(SupervisedModels, prettyNames, 
#                        X=trainingData.drop(["material_id", "full_formula"], axis=1),
#                       k = numberSplits, n = numberRuns)

In [ ]:
visualize.plot_important_features_restricted_domain(SupervisedModels, prettyNames, trainingData, k = numberSplits, n = numberRuns)

## Precision and recall metrics

In [ ]:
def plot_confusion_metrics(models, names, data, abbreviations=[]):
    fig = go.Figure( 
            layout = go.Layout (
                title=go.layout.Title(text="False positives (Nruns = {})".format(numberRuns*numberSplits)),
                yaxis=dict(title='Counts'),
                barmode='group'
            )
        )

    for i, model in enumerate(models):

        fig.add_traces(go.Bar(name=names[i], 
                            x=data['full_formula'][model['falsePositives'] > 0],
                            y=model['falsePositives'][model['falsePositives'] > 0]))

    fig.show()

    fig = go.Figure( 
            layout = go.Layout (
                title=go.layout.Title(text="False negatives (Nruns = {})".format(numberRuns*numberSplits)),
                yaxis=dict(title='Counts'),
                barmode='group'
            )
        )
    for i, model in enumerate(models):
        fig.add_traces(go.Bar(name=names[i], 
                                x=data['full_formula'][model['falseNegatives'] > 0],
                                y=model['falseNegatives'][model['falseNegatives'] > 0]))

    fig.show()
plot_confusion_metrics(SupervisedModels, prettyNames, trainingData)

In [ ]:
visualize.plot_confusion_matrixQT(SupervisedModels, trainingTarget, trainingData, names=prettyNames, k = numberSplits, n = numberRuns)

In [ ]:
visualize.confusion_matrixQT(SupervisedModels, trainingTarget, prettyNames)

# Predicting solid-state qubit candidates

It is time to make the prediction based on the best estimators and features possible. We have chosen to choose the features that have been deemed important by sklearn at least 50 percent of the cross validation runs as important. 

In [ ]:
Summary                 = pd.DataFrame({}, dtype="string")
Summary["material_id"]  = testSet["material_id"]
Summary["full_formula"] = testSet["full_formula"]
Summary["pretty_formula"] = testSet["pretty_formula"]

PredictedCandidates = pd.Series({}, dtype="string")

threshold = numberSplits*numberRuns/2 #50% when equal
trainSet = trainingData.drop(["material_id", "full_formula"], axis=1)
testData = testSet.drop(["pretty_formula", "candidate", "full_formula", "material_id"], axis=1)
fittedAlgorithms = [] 

for i, algorithm in tqdm(enumerate(Algorithms)):
    
    fittedAlgorithm = train_model.fitAlgorithm(algorithm, 
                                    trainingData   = trainSet[trainSet.columns[SupervisedModels[Abbreviations[i]]["importantKeys"]>threshold]],\
                                    trainingTarget = trainingTarget.values.reshape(-1,),)
    
    fittedAlgorithms.append(fittedAlgorithm)
    
    PredictedCandidates[Abbreviations[i]],\
    PredictedCandidates[Abbreviations[i]+" Prob"] = predict_model.runPredictions(fittedAlgorithm,\
                                                        testData = testData[testData.columns[SupervisedModels[Abbreviations[i]]["importantKeys"]>threshold]])

In [ ]:
for abbreviation in Abbreviations:
    Summary[abbreviation]            = PredictedCandidates[abbreviation]
    Summary[abbreviation + "Prob"]       = PredictedCandidates[abbreviation + "Prob"]
    print("{} predict the number of candidates as: {}".format(abbreviation, int(np.sum(PredictedCandidates[abbreviation]))))

In [ ]:
Summary

## Save the summary and models

In [ ]:
for i, fitted_algorithm in tqdm(enumerate(fittedAlgorithms)):
    file_path = Path(models_dir / InsertApproach / "trained-models" / Path(prettyNames[i] + ".pkl"))
    with file_path.open("wb") as fp:
        pickle.dump(fitted_algorithm, fp)
        

Summary.to_pickle(models_dir / InsertApproach / "summary" / "summary.pkl")